In [1]:
import pandas as pd
import os

# 1. Set base paths (works in a notebook)
BASE_DIR = os.path.abspath("..")  # parent folder of /notebooks
csv_path = os.path.join(BASE_DIR, "data", "processed", "cpi_food_clean.csv")

# 2. Load the cleaned CPI data
df = pd.read_csv(csv_path)

# 3. Convert date column to datetime
df["date"] = pd.to_datetime(df["date"])

# 4. Sort by category + date so time-series is in order
df = df.sort_values(["category", "date"])

# 5. Calculate Month-over-Month % change for each category
df["mom_pct_change"] = (
    df.groupby("category")["value"].pct_change() * 100
)

# 6. Quick preview
df.head()


,date,series_id,category,value,mom_pct_change
0,2015-01-01,CUSR0000SEFC,Cereals & bakery,325.728,NaN
1,2015-02-01,CUSR0000SEFC,Cereals & bakery,326.521,0.243455
2,2015-03-01,CUSR0000SEFC,Cereals & bakery,324.972,-0.474395
3,2015-04-01,CUSR0000SEFC,Cereals & bakery,325.895,0.284024
4,2015-05-01,CUSR0000SEFC,Cereals & bakery,324.713,-0.362694


In [2]:
# Step 2: Remove unrealistic MoM spikes (economic sanity check)
df_clean = df[df["mom_pct_change"].between(-20, 20)]

df_clean.head()

,date,series_id,category,value,mom_pct_change
1,2015-02-01,CUSR0000SEFC,Cereals & bakery,326.521,0.243455
2,2015-03-01,CUSR0000SEFC,Cereals & bakery,324.972,-0.474395
3,2015-04-01,CUSR0000SEFC,Cereals & bakery,325.895,0.284024
4,2015-05-01,CUSR0000SEFC,Cereals & bakery,324.713,-0.362694
5,2015-06-01,CUSR0000SEFC,Cereals & bakery,325.447,0.226046


In [3]:
# Step 3: Remove categories known to produce unstable MoM values
unstable_categories = [
    "Cereals & bakery",
    "Nonalcoholic beverages"
]

df_clean = df_clean[~df_clean["category"].isin(unstable_categories)]

df_clean.head()


,date,series_id,category,value,mom_pct_change
121,2015-02-01,CUSR0000SEFP,Dairy,121.375,0.139432
122,2015-03-01,CUSR0000SEFP,Dairy,121.028,-0.285891
123,2015-04-01,CUSR0000SEFP,Dairy,121.208,0.148726
124,2015-05-01,CUSR0000SEFP,Dairy,119.916,-1.065936
125,2015-06-01,CUSR0000SEFP,Dairy,120.276,0.300210


In [4]:
# Step 4: Remove rows with unusually small CPI values
df_clean = df_clean[df_clean["value"] > 50]

df_clean.head()

,date,series_id,category,value,mom_pct_change
121,2015-02-01,CUSR0000SEFP,Dairy,121.375,0.139432
122,2015-03-01,CUSR0000SEFP,Dairy,121.028,-0.285891
123,2015-04-01,CUSR0000SEFP,Dairy,121.208,0.148726
124,2015-05-01,CUSR0000SEFP,Dairy,119.916,-1.065936
125,2015-06-01,CUSR0000SEFP,Dairy,120.276,0.300210


In [5]:
# Step 5: Save the cleaned MoM dataset
clean_mom_path = os.path.join(BASE_DIR, "data", "processed", "cpi_food_mom_clean.csv")
df_clean.to_csv(clean_mom_path, index=False)

print("Clean MoM dataset saved at:", clean_mom_path)

Clean MoM dataset saved at: /Users/udayb/Desktop/Portfolio/food-inflation-insights/data/processed/cpi_food_mom_clean.csv
